# Lab 03: Feature Engineering for Regression using Short Rental Dataset


**Objective:**
In this lab, you will extract and process different types of features (numerical, categorical, and textual) from the provided datasets (`train.csv.gz`, `test.csv.gz`) for your Program 2 assignment. You will construct feature vectors to use in a regression model to predict the price of listings.

The datasets are on the HPC, under `/WAVE/projects/CSEN-140-Sp25/data/pr2`

In [56]:

# Import required libraries
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Load datasets
train_df = pd.read_csv("train.csv.gz", compression='gzip')
test_df = pd.read_csv("test.csv.gz", compression='gzip')

# Show a sample of the datasets
print("Train Data Sample")
display(train_df.head())

print("Test Data Sample")
display(test_df.head())

Train Data Sample


,name,description,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,...,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,availability_eoy,number_of_reviews_ly,estimated_occupancy_l365d,estimated_revenue_l365d,reviews,price
0,1 bedroom apt 10469,You'll have a great time at this comfortable p...,Jose,2019-10-01,"New York, NY",NaN,within an hour,86%,81%,t,...,5,0,0,NaN,271.0,0.0,0.0,0.0,NaN,70.0
1,Spacious 3x2 for Visiting ATX,"This is a 3 bedroom, 2 bathroom condo in the h...",Susana,2019-07-17,"Austin, TX",Hey y'all! My name is Susana & I'm so happy th...,within an hour,96%,100%,f,...,47,0,0,1.08,0.0,1.0,18.0,1584.0,NaN,88.0
2,Spacious place Palm Culver City,Bring the whole family to this great place wit...,Dee,2012-07-30,"Los Angeles, CA",I am a mental health professional in addition ...,within an hour,100%,99%,f,...,21,0,0,NaN,304.0,0.0,0.0,0.0,NaN,130.0
3,2-bedroom Mission Beach home with private patio,"Cute 2-bedroom, 1-bath, downstairs unit in dup...",Tracy,2014-01-14,"San Diego, CA",NaN,within an hour,100%,100%,t,...,3,0,0,2.72,NaN,NaN,NaN,NaN,Great place to stay with little caveats. Ye wa...,231.0
4,334-cozy apt 5 mins to beach,Discover comfort in Prime Fort Lauderdale Loca...,Michael,2024-06-06,"Hollywood, FL","Hello! \nI'm a proud Floridian, having lived h...",within an hour,100%,67%,t,...,16,2,0,1.00,NaN,NaN,NaN,NaN,NaN,129.0


Test Data Sample


,name,description,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,...,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,availability_eoy,number_of_reviews_ly,estimated_occupancy_l365d,estimated_revenue_l365d,reviews
0,Luxury private room near JFK Int. Airport New ...,Luxury spacious private room with attached ful...,Mohammed,2022-07-24,"New York, NY",Knowledge Comes Before Speech and Action ( الع...,within an hour,100%,93%,t,...,20,1,19,0,0.50,305.0,3.0,180.0,8460.0,NaN
1,Beautiful resort 1 bedroom full kitchen,Make some memories at this unique and family-f...,Ronnie,2019-08-30,NaN,Love people and loves God,within a day,100%,0%,f,...,2,2,0,0,0.05,NaN,NaN,NaN,NaN,NaN
2,Specious large alcove studio,Super large specious studio loft style located...,Yael,2015-08-04,"New York, NY",NaN,within an hour,100%,100%,f,...,17,17,0,0,0.08,276.0,1.0,0.0,0.0,NaN
3,Quiet bedroom with private bathroom,"4 bedroom home, 3 upstairs, 1 downstairs. Mod...",Natallia,2012-02-19,"San Francisco, CA",We enjoy the Airbnb experience--what cultures ...,within an hour,100%,100%,f,...,5,2,3,0,0.63,136.0,10.0,255.0,21420.0,Highly recommend the property! The Sonder was ...
4,Sunny Good Vibes with View & Work from Home,Welcome to Sunny Good Vibes in the historic Mi...,Garret,2017-04-27,United States,I’m the owner and operator of Sunny Good Vibes...,within an hour,100%,99%,t,...,2,2,0,0,3.05,NaN,NaN,NaN,NaN,NaN


## Step 1: Data Preprocessing

In [57]:
# Check for missing values in train and test sets
print(train_df.isnull().sum())
print(test_df.isnull().sum())

name                             0
description                   1608
host_name                      113
host_since                     115
host_location                20394
                             ...  
number_of_reviews_ly         29176
estimated_occupancy_l365d    29176
estimated_revenue_l365d      29176
reviews                      85757
price                            0
Length: 66, dtype: int64
name                             0
description                    382
host_name                       33
host_since                      34
host_location                 4962
                             ...  
availability_eoy              7140
number_of_reviews_ly          7140
estimated_occupancy_l365d     7140
estimated_revenue_l365d       7140
reviews                      21441
Length: 65, dtype: int64


In [58]:

# Decide which columns to drop or fill in with default values based on your analysis
# Should you do it based on training data or test data statistics, or both?
# Answer: It should only be based on the training data. Otherwise you risk overfitting to just the test data and nothing else.

# If you want to sample instead uncomment these lines. But the whole thing works without sampling.
# train_df = train_df.sample(n=10000, random_state=42).reset_index(drop=True)
# test_df = test_df.sample(n=10000, random_state=42).reset_index(drop=True)

categorical_features = [
    'property_type',
    'room_type',
    'neighbourhood_cleansed',
    'neighbourhood_group_cleansed',
    'host_response_time',
    'host_is_superhost',
    'host_has_profile_pic',
    'host_identity_verified',
    'has_availability',
    'instant_bookable',
    'calendar_updated',
    'host_name',
    'host_location',
    'host_neighbourhood',
    'license',
    'host_verifications'
]

text_features = [
    'name',
    'description',
    'host_about',
    'reviews',
    'bathrooms_text',
    'amenities'
]

numerical_features = [
    'accommodates',
    'bathrooms',
    'bedrooms',
    'beds',
    'host_listings_count',
    'host_total_listings_count',
    'availability_30',
    'availability_60',
    'availability_90',
    'availability_365',
    'minimum_nights',
    'maximum_nights',
    'minimum_minimum_nights',
    'maximum_minimum_nights',
    'minimum_maximum_nights',
    'maximum_maximum_nights',
    'minimum_nights_avg_ntm',
    'maximum_nights_avg_ntm',
    'reviews_per_month',
    'number_of_reviews',
    'number_of_reviews_ltm',
    'number_of_reviews_l30d',
    'review_scores_rating',
    'review_scores_accuracy',
    'review_scores_cleanliness',
    'review_scores_checkin',
    'review_scores_communication',
    'review_scores_location',
    'review_scores_value',
    'estimated_occupancy_l365d',
    'estimated_revenue_l365d',
    'availability_eoy',
    'calculated_host_listings_count',
    'calculated_host_listings_count_entire_homes',
    'calculated_host_listings_count_private_rooms',
    'calculated_host_listings_count_shared_rooms',
    'host_response_rate',
    'host_acceptance_rate',
    'number_of_reviews_ly'
]

remove_features = [
    'calendar_updated',
    'license',
    'host_about',
    'reviews',
    'host_location',
    'neighbourhood',
    'neighbourhood_cleansed',
    'neighbourhood_group_cleansed',
    'host_name',
    'bathrooms_text',
    'host_since',
    'first_review',
    'last_review',
]

categorical_features = [f for f in categorical_features if f not in remove_features]
numerical_features   = [f for f in numerical_features if f not in remove_features]
text_features = [f for f in text_features if f not in remove_features]

train_df = train_df.drop(columns=remove_features)
test_df = test_df.drop(columns=remove_features)

def clean_numeric_column(df, col):
    # Convert percentages (e.g. '86%') and text numbers (e.g. '2 baths')
    df[col] = df[col].astype(str).str.extract(r'([\d.]+)').astype(float)
    return df

# Fill in missing numerical values with column averages
for col in numerical_features:
    if train_df[col].isnull().any():
        train_df = clean_numeric_column(train_df, col)
        mean_value = train_df[col].mean()
        train_df[col] = train_df[col].fillna(mean_value)
    
    if test_df[col].isnull().any():
        test_df = clean_numeric_column(test_df, col)
        # Use the same mean as from the training set to avoid leakage
        test_df[col] = test_df[col].fillna(mean_value)

for col in text_features:
    train_df[col] = train_df[col].fillna('').astype(str)
    test_df[col] = test_df[col].fillna('').astype(str)
    
for col in categorical_features:
    train_df[col] = train_df[col].astype(str).fillna('missing')
    test_df[col] = test_df[col].astype(str).fillna('missing')

columns_to_join = ['amenities', 'host_verifications']

for col in columns_to_join:
    train_df[col] = train_df[col].apply(lambda x: " ".join(x) if isinstance(x, list) else str(x))
    test_df[col] = test_df[col].apply(lambda x: " ".join(x) if isinstance(x, list) else str(x))


## Step 2: Feature Extraction

In [59]:

# Numerical columns should likely be standardized
# Categorical columns should be one-hot encoded or label encoded
# Text columns should be vectorized (e.g., using TF-IDF)

# Define categorical and text features
#categorical_features = ['room_type', 'host_response_time', 'host_is_superhost', # etc (many more categorical features)

# Standardize numerical features - here's an example using StandardScaler; You can choose other options too
scaler = StandardScaler()
train_df[numerical_features] = scaler.fit_transform(train_df[numerical_features])
test_df[numerical_features] = scaler.transform(test_df[numerical_features])


In [60]:

# You may want to one-hot encode categorical features (I'll let you figure out how to do this)
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoded_train = encoder.fit_transform(train_df[categorical_features])
encoded_test = encoder.transform(test_df[categorical_features])
encoded_train_df = pd.DataFrame(encoded_train, columns=encoder.get_feature_names_out(categorical_features), index=train_df.index)
encoded_test_df = pd.DataFrame(encoded_test, columns=encoder.get_feature_names_out(categorical_features), index=test_df.index)

# Drop the old categorical columns
train_df = train_df.drop(columns=categorical_features)
test_df = test_df.drop(columns=categorical_features)

# Concatenate encoded columns
train_df = pd.concat([train_df, encoded_train_df], axis=1)
test_df = pd.concat([test_df, encoded_test_df], axis=1)


In [61]:
# And figure out what to do with text features (e.g., TF-IDF vectorization, key word extraction, sentiment analysis, etc.)
# You could use TfidfVectorizer for text features, which does the same thing we did in lab02, or take a different approach
# Remember to do some text preprocessing (remove punctuation, lowercase, etc.)

import string
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from pandas.api.types import CategoricalDtype

def remove_punctuation(s):
    """
    Remove punctuation from a string
    """
    return s.translate(str.maketrans('','', string.punctuation))

def preprocess_line(line):
    stop_words = set(stopwords.words('english'))
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    return [
                lemmatizer.lemmatize(stemmer.stem(w.lower())) 
                for w in remove_punctuation(line).split() 
                if len(w) > 0 and w.lower() not in stop_words
        ] 

def process_train_data_text(df, tokenizer):
    # Create a (name, transformer, column) tuple for each text feature
    text_transformers = [
        (f"{col}_tfidf", TfidfVectorizer(
            tokenizer=tokenizer,
            preprocessor=None,
            lowercase=False,
            stop_words=None,
            max_features = 500
        ), col)
        for col in text_features
    ]

    # Combine all TF-IDF transformers
    text_pipeline = ColumnTransformer(transformers=text_transformers)

    # Fit and transform the training data
    train_text = text_pipeline.fit_transform(df)

    return train_text, text_pipeline

train_text, text_pipeline = process_train_data_text(train_df, preprocess_line)
test_text = text_pipeline.transform(test_df)

column_names = text_pipeline.get_feature_names_out()

df_text_train = pd.DataFrame(train_text.toarray(), columns=column_names, index=train_df.index)
df_text_test = pd.DataFrame(test_text.toarray(), columns=column_names, index=test_df.index)

train_df = pd.concat([train_df.drop(columns=text_features), df_text_train], axis=1)
test_df = pd.concat([test_df.drop(columns=text_features), df_text_test], axis=1)

/WAVE/users2/unix/avattuone/.conda/envs/140wi25/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


## Step 4: Put All Features Together or Decide How to Process the Samples

In [70]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Example below assumes your train_df and test_df only contain numerical features

# Split the data into features and target variable
X_train = train_df.drop(columns=['price'])  # Drop the target variable
y_train = train_df['price']  # Target variable
X_test = test_df

# Example regression model
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test) 

y_train_pred = model.predict(X_train)

rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
print(f"Root Mean Squared Error on Training Data: {rmse}")

Root Mean Squared Error on Training Data: 732.2260949255975


In [78]:
# Extra Statistics Displayed
Q1 = y_train.quantile(0.25)
Q3 = y_train.quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter y_train and y_train_pred using mask
mask_train = (y_train >= lower_bound) & (y_train <= upper_bound)
y_train_filtered = y_train[mask_train]
y_train_pred_filtered = y_train_pred[mask_train]

# Filter y_pred using same IQR logic
mask_test = (y_pred >= lower_bound) & (y_pred <= upper_bound)
y_pred_filtered = y_pred[mask_test]
X_test_filtered = X_test[(y_pred >= lower_bound) & (y_pred <= upper_bound)]

# Wrap all arrays in Series for clean describe output
y_train_series = pd.Series(y_train, index=y_train.index, name="y_train")
y_train_pred_series = pd.Series(y_train_pred, index=y_train.index, name="y_train_pred")
y_pred_series = pd.Series(y_pred, index=X_test.index, name="y_pred")

y_train_filtered_series = pd.Series(y_train_filtered, index=y_train_filtered.index, name="y_train_filtered")
y_train_pred_filtered_series = pd.Series(y_train_pred_filtered, index=y_train_filtered.index, name="y_train_pred_filtered")
y_pred_filtered_series = pd.Series(y_pred_filtered, index=X_test_filtered.index, name="y_pred_filtered")

# Calculate RMSE
rmse_with_outliers = np.sqrt(mean_squared_error(y_train_series, y_train_pred_series))
rmse_no_outliers = np.sqrt(mean_squared_error(y_train_filtered_series, y_train_pred_filtered_series))

# Output RMSE values
print(f"Root Mean Squared Error (With Outliers): {rmse_with_outliers:.4f}")
print(f"Root Mean Squared Error (Removed Outliers): {rmse_no_outliers:.4f}\n")

# Print descriptive statistics
print("=== TRAINING STATS (WITH OUTLIERS) ===")
print("Actual:")
print(y_train_series.describe())
print("\nPredicted:")
print(y_train_pred_series.describe())

print("\n=== TRAINING STATS (FILTERED) ===")
print("Actual:")
print(y_train_filtered_series.describe())
print("\nPredicted:")
print(y_train_pred_filtered_series.describe())

print("\n=== TEST SET STATS (WITH OUTLIERS) ===")
print("Predicted:")
print(y_pred_series.describe())

print("\n=== TEST SET STATS (FILTERED) ===")
print("Predicted:")
print(y_pred_filtered_series.describe())

Root Mean Squared Error (With Outliers): 732.2261
Root Mean Squared Error (Removed Outliers): 213.1175

=== TRAINING STATS (WITH OUTLIERS) ===
Actual:
count     89524.000000
mean        283.773793
std         841.794875
min           7.000000
25%          98.000000
50%         161.000000
75%         282.000000
max      100000.000000
Name: y_train, dtype: float64

Predicted:
count    89524.000000
mean       283.773793
std        415.280109
min      -1691.824500
25%         76.139484
50%        205.198330
75%        381.326485
max       9250.912194
Name: y_train_pred, dtype: float64

=== TRAINING STATS (FILTERED) ===
Actual:
count    81809.000000
mean       180.018653
std        115.954135
min          7.000000
25%         93.000000
50%        150.000000
75%        239.000000
max        558.000000
Name: y_train_filtered, dtype: float64

Predicted:
count    81809.000000
mean       216.227630
std        244.455969
min      -1691.824500
25%         64.127355
50%        183.013297
75%       